In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from dateutil.relativedelta import relativedelta
import utils
import cf_xarray as cfxr

from sklearn.linear_model import LinearRegression
import scipy.signal as signal
from scipy import stats
from datetime import timedelta

import seaborn as sns
import matplotlib as mpl
import cmocean
import cmocean.cm as cmo
from matplotlib.gridspec import GridSpec

from matplotlib.lines import Line2D
import matplotlib.patches as mpatches

import string

In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': True})

## Import G

In [3]:
G_ds = xr.open_dataset('Outputs/G_ds.nc4')['__xarray_dataarray_variable__']

G_CDR_ds = xr.open_dataset('Outputs/G_cdr_ds.nc4')['__xarray_dataarray_variable__']


In [4]:
#rolling 5 year mean 
G_ds = G_ds.rolling(year = 5, min_periods = 1, center = False).mean('year')
G_CDR_ds = G_CDR_ds.rolling(year = 5, min_periods = 1, center = False).mean('year')

In [5]:
G_ds = xr.concat([G_ds, -G_CDR_ds], pd.Index(['pulse','cdr'], name = 'pulse_type'))


In [6]:
G_ds.name = 'G[tas]'
G_ds = G_ds.rename({'year':'s'})


## Notes:

GFDL: 1pct and esm pi-control start from year 0001

UKESM1: 1pct starts in 1850 and pi-control starts in 1960, move 1pct to start in 1960

MIROC: both start from 1850

NORESM2: 1pct from 0001 pi-control from 1600-- move 1pct to 1600

ACCESS: 1pct and pi-control from 0101

CANESM5_r1p2: 1pct 1850, pi-control 5550, move 1pct to 5550


In [7]:
model_run_pulse_dict = utils.model_run_pulse_dict
model_run_1pct_dict = utils.model_run_1pct_dict
model_run_control_dict = utils.model_run_picontrol_dict
model_run_cdr_pulse_dict = utils.model_run_cdr_pulse_dict
model_run_1pct1000gtc_dict = utils.model_run_1pct_1000gtc_dict


In [8]:
model_color = utils.model_color
type_color = utils.type_color

In [9]:
A = utils.A
ds_out = utils.ds_out

In [10]:
tas_co2_1pct1000gtc = {}
tas_co2_pictrl = {}
tas_1pct = {}

for m in model_run_1pct_dict.keys():
    print(m)
    print('tas')
    tas_1pct[m] = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_1pct_dict[m]}',  use_cftime=True) #kg/m2/s
    lat_corners = cfxr.bounds_to_vertices(tas_1pct[m].isel(time = 0)['lat_bnds'], "bnds", order=None)
    lon_corners = cfxr.bounds_to_vertices(tas_1pct[m].isel(time = 0)['lon_bnds'], "bnds", order=None)
    tas_1pct[m] = tas_1pct[m].assign(lon_b=lon_corners, lat_b=lat_corners)
    tas_1pct[m] = utils._regrid_ds(tas_1pct[m], ds_out)
    
    
for m in model_run_1pct1000gtc_dict.keys():
    print(m)
    print('tas')
    tas_co2_1pct1000gtc[m] = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_1pct1000gtc_dict[m]}',  use_cftime=True) #kg/m2/s
    lat_corners = cfxr.bounds_to_vertices(tas_co2_1pct1000gtc[m].isel(time = 0)['lat_bnds'], "bnds", order=None)
    lon_corners = cfxr.bounds_to_vertices(tas_co2_1pct1000gtc[m].isel(time = 0)['lon_bnds'], "bnds", order=None)
    tas_co2_1pct1000gtc[m] = tas_co2_1pct1000gtc[m].assign(lon_b=lon_corners, lat_b=lat_corners)
    tas_co2_1pct1000gtc[m] = utils._regrid_ds(tas_co2_1pct1000gtc[m], ds_out)


UKESM1_r1
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


UKESM1_r2
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


UKESM1_r3
tas


...

/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


CANESM5_r2p2
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


CANESM5_r3p2
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


In [11]:
for m in tas_co2_1pct1000gtc.keys():
    tas_co2_1pct1000gtc[m] = xr.concat([tas_1pct[m].loc[dict(time = slice(tas_1pct[m]['time'].min(), tas_co2_1pct1000gtc[m]['time'].min()-timedelta(30)))] , 
                                        tas_co2_1pct1000gtc[m]],
                 dim = 'time')


In [12]:
for m in model_run_control_dict.keys():
    print(m)
    print('tas')
    tas_co2_pictrl[m] = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_control_dict[m]}',  use_cftime=True) #kg/m2/s
    lat_corners = cfxr.bounds_to_vertices(tas_co2_pictrl[m].isel(time = 0)['lat_bnds'], "bnds", order=None)
    lon_corners = cfxr.bounds_to_vertices(tas_co2_pictrl[m].isel(time = 0)['lon_bnds'], "bnds", order=None)
    tas_co2_pictrl[m] = tas_co2_pictrl[m].assign(lon_b=lon_corners, lat_b=lat_corners)
    tas_co2_pictrl[m] = utils._regrid_ds(tas_co2_pictrl[m], ds_out)

## fix the times so that they line up according to the notes above
m = 'NORESM2'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl[m]['time'] -timedelta(365*1599)

m = 'UKESM1_r1'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl[m]['time'] - timedelta(360*110)

m = 'CANESM5_r1p2'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl[m]['time']- timedelta(365*3700)

m = 'CANESM5_r1p1'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl[m]['time']- timedelta(365*3351)

UKESM1_r1
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


MIROC
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


NORESM2
tas


...

/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


CANESM5_r1p2
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


CANESM5_r1p1
tas


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


In [13]:
dif_1000gtc = {}
for m1 in model_run_1pct1000gtc_dict.keys():
    print(m1)
    if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
        m2 = 'UKESM1_r1'
    elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2' or m1 == 'CANESM5_r4p2' or m1 == 'CANESM5_r5p2':
         m2 = 'CANESM5_r1p2'
    else:
        m2 = m1
    print(m1, m2)
    
    dif_1000gtc[m1] = tas_co2_1pct1000gtc[m1] - tas_co2_pictrl[m2]
    
    if len(dif_1000gtc[m1]['time']) > 3000:  #hack to get the time stamping to work, should find better fix
        periods = 3000
    else:
        periods = len(dif_1000gtc[m1]['time'])
        
    times = pd.date_range('2000', periods= periods, freq='MS')
    weights = times.shift(1, 'MS') - times
    weights = xr.DataArray(weights, [('time', dif_1000gtc[m1]['time'][:periods].values)]).astype('float')
    dif_1000gtc[m1] =  (dif_1000gtc[m1] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')

    dif_1000gtc[m1]['year'] = range(len(dif_1000gtc[m1]['year']))

UKESM1_r1
UKESM1_r1 UKESM1_r1
UKESM1_r2
UKESM1_r2 UKESM1_r1
UKESM1_r3
UKESM1_r3 UKESM1_r1
UKESM1_r4
UKESM1_r4 UKESM1_r1
MIROC
MIROC MIROC
NORESM2
NORESM2 NORESM2
ACCESS
ACCESS ACCESS
CANESM5_r1p2
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r2p2
CANESM5_r2p2 CANESM5_r1p2
CANESM5_r3p2
CANESM5_r3p2 CANESM5_r1p2


In [14]:
dif_1pct = {}
for m1 in model_run_1pct_dict.keys():
    if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
        m2 = 'UKESM1_r1'
    elif m1 == 'CANESM5_r1p1' or m1 == 'CANESM5_r2p1' or m1 == 'CANESM5_r3p1':
         m2 = 'CANESM5_r1p1'
    elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2':
         m2 = 'CANESM5_r1p2'
    else:
        m2 = m1
    print(m1, m2)
    
    
    dif_1pct[m1] = tas_1pct[m1] - tas_co2_pictrl[m2]
    
    if len(dif_1pct[m1]['time']) > 3000:  #hack to get the time stamping to work, should find better fix
        periods = 3000
    else:
        periods = len(dif_1pct[m1]['time'])
        
    times = pd.date_range('2000', periods= periods, freq='MS')
    weights = times.shift(1, 'MS') - times
    weights = xr.DataArray(weights, [('time', dif_1pct[m1]['time'][:periods].values)]).astype('float')
    dif_1pct[m1] =  (dif_1pct[m1] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')

    dif_1pct[m1]['year'] = range(len(dif_1pct[m1]['year']))

UKESM1_r1 UKESM1_r1
UKESM1_r2 UKESM1_r1
UKESM1_r3 UKESM1_r1
UKESM1_r4 UKESM1_r1
MIROC MIROC
NORESM2 NORESM2
ACCESS ACCESS
GFDL GFDL
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r2p2 CANESM5_r1p2
CANESM5_r3p2 CANESM5_r1p2
CANESM5_r1p1 CANESM5_r1p1
CANESM5_r2p1 CANESM5_r1p1
CANESM5_r3p1 CANESM5_r1p1


In [15]:
for m in dif_1pct.keys():
    dif_1pct[m] = dif_1pct[m].drop('height')
for m in dif_1000gtc.keys():
    dif_1000gtc[m] = dif_1000gtc[m].drop('height')

In [16]:
#get rid of height and limit the time to the length of the GF
for m1 in ['UKESM1_r1', 'UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2',
       'GFDL', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2']:
    
    for t in ['pulse','cdr']:
        if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
            m2 = 'UKESM1_r1'
        elif m1 == 'CANESM5_r1p1' or m1 == 'CANESM5_r2p1' or m1 == 'CANESM5_r3p1':
             m2 = 'CANESM5_r1p1'
        elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2':
             m2 = 'CANESM5_r1p2'
        else:
            m2 = m1
    
        
        length = len(G_ds.sel(model = m2, pulse_type = t).dropna(dim = 's')['s'])
        dif_1pct[m] = dif_1pct[m].sel(year = slice(0,length))


In [17]:
for m1 in ['UKESM1_r1', 'UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2',
       'GFDL', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2']:
    
    for t in ['pulse','cdr']:
        if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
            m2 = 'UKESM1_r1'
        elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2' or m1 == 'CANESM5_r4p2' or m1 == 'CANESM5_r5p2':
             m2 = 'CANESM5_r1p2'
        else:
            m2 = m1    
        
        length = len(G_ds.sel(model = m2, pulse_type = t).dropna(dim = 's')['s'])
        dif_1000gtc[m] = dif_1000gtc[m].sel(year = slice(0,10))

In [18]:
ds_dif_1pct = xr.concat([dif_1pct[m] for m in dif_1pct.keys()], pd.Index([m for m in dif_1pct.keys()], name='model'), coords='minimal')


In [19]:
ds_dif_1000gtc = xr.concat([dif_1000gtc[m] for m in dif_1000gtc.keys()], pd.Index([m for m in dif_1000gtc.keys()], name='model'), coords='minimal')


In [20]:
ds_dif = xr.concat([ds_dif_1000gtc, ds_dif_1pct], pd.Index(['1000gtc','1pct'], name='experiment'), coords='minimal')


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/dask/array/core.py:4697: PerformanceWarning: Increasing number of chunks by factor of 10
  result = blockwise(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/dask/array/core.py:4697: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(


In [21]:
ds_dif = ds_dif.rename({'year':'s'})

## Emissions profile

In [22]:
emis_profile_1pct = xr.open_dataset(f'Outputs/1pct_emis_profile_full.nc4')
emis_profile_1pct = emis_profile_1pct.rename({'__xarray_dataarray_variable__':'emis'})

emis_profile_1000gtc =  xr.open_dataset(f'Outputs/1pct1000gtc_emis_profile_full.nc4')
emis_profile_1000gtc = emis_profile_1000gtc.rename({'__xarray_dataarray_variable__':'emis'})

In [23]:
emis_profile = xr.concat([emis_profile_1000gtc, emis_profile_1pct], pd.Index(['1000gtc','1pct'], name='experiment'), coords='minimal')


## Define our Model Weights

In [24]:
#define our weights for models (grouping UKESM and CANESM realizations)
model_weights = {'UKESM1_r1': 0.25, 'UKESM1_r2': 0.25, 'UKESM1_r3': 0.25, 'UKESM1_r4': 0.25, 'NORESM2': 1, 'GFDL': 1,
       'MIROC': 1, 'ACCESS': 1,  'CANESM5_r2p2':1/3, 'CANESM5_r1p2':1/3, 'CANESM5_r3p2':1/3}
model_weights = xr.DataArray(
    data=list(model_weights.values()),
    dims=["model"],
    coords=dict(
        model=(["model"], list(model_weights.keys()))
    ),
    attrs=dict(
        description="weights for models"
    ),
)

In [25]:
#define our weights for models (grouping UKESM and CANESM realizations)
onepct_model_weights = {'UKESM1_r1': 0.25, 'UKESM1_r2': 0.25, 'UKESM1_r3': 0.25, 'UKESM1_r4': 0.25, 'NORESM2': 1, 'GFDL': 1,
       'MIROC': 1, 'CANESM5_r3p1':1/6, 'ACCESS':1, 'CANESM5_r2p2':1/6, 'CANESM5_r2p1':1/6,
       'CANESM5_r1p2':1/6, 'CANESM5_r1p1':1/6, 'CANESM5_r3p2':1/6}
onepct_model_weights = xr.DataArray(
    data=list(onepct_model_weights.values()),
    dims=["model"],
    coords=dict(
        model=(["model"], list(onepct_model_weights.keys()))
    ),
    attrs=dict(
        description="weights for models"
    ),
)

In [26]:
G_model_weights = {'UKESM1_r1': 1, 'NORESM2': 1, 'GFDL': 1,
       'MIROC': 1, 'ACCESS': 1,  'CANESM5_r1p2':1/3, 'CANESM5_r2p2':1/3, 'CANESM5_r3p2':1/3}
G_model_weights = xr.DataArray(
    data=list(G_model_weights.values()),
    dims=["model"],
    coords=dict(
        model=(["model"], list(G_model_weights.keys()))
    ),
    attrs=dict(
        description="weights for Green's function"
    ),
)

## PiCtrl

In [27]:

#combine our picontrol data into one dataset, normalizing the time to year 0
pictrl = {}
for m in tas_co2_pictrl.keys():    
    times = tas_co2_pictrl[m].time.get_index('time')
    weights = times.shift(-1, 'MS') - times.shift(1, 'MS')
    weights = xr.DataArray(weights, [('time', tas_co2_pictrl[m]['time'].values)]).astype('float')
    pictrl[m] =  (tas_co2_pictrl[m] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')
    pictrl[m]['year'] = pictrl[m]['year'] - pictrl[m]['year'][0] 
    
for m in pictrl.keys():
    pictrl[m] = pictrl[m].drop('height')
ds_pictrl = xr.concat([pictrl[m] for m in pictrl.keys()], pd.Index([m for m in pictrl.keys()], name='model'), coords='minimal')


## Global Mean Analysis

In [ ]:
%%time
GF = G_ds.weighted(A).mean(dim = ['lat','lon'])

conv_mean = {}
for exp in ['1000gtc','1pct']:
    conv_mean[exp] = {}
    for m1 in ['UKESM1_r1','UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2', 'ACCESS',
       'GFDL', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'CANESM5_r3p2']:
        conv_mean[exp][m1] = {}
        for t in ['pulse','cdr']:
            if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
                m2 = 'UKESM1_r1'
            else:
                m2 = m1
            conv_mean[exp][m1][t] = signal.convolve( np.array(GF.sel(model = m2, pulse_type = t).dropna(dim = 's')), np.array(emis_profile.sel(model = m1, experiment = exp)['emis']),'full')
            conv_mean[exp][m1][t] = utils.np_to_xr_mean(conv_mean[exp][m1][t], GF.sel(model = m2, pulse_type = t), emis_profile.sel(model = m1, experiment = exp))
            length = len(G_ds.weighted(A).mean(dim = ['lat','lon']).dropna(dim = 's')['s'])
            conv_mean[exp][m1][t] = conv_mean[exp][m1][t][:length]


In [ ]:
#convert to dataset

conv_dict = {}
for exp in conv_mean.keys():
    conv_dict[exp] = {}
    for m in conv_mean[exp].keys():
        conv_dict[exp][m] = xr.concat([conv_mean[exp][m][t] for t in conv_mean[exp][m].keys()], pd.Index([t for t in conv_mean[exp][m].keys()], name='pulse_type'), coords='minimal')
for exp in conv_mean.keys():
    conv_dict[exp] = xr.concat([conv_dict[exp][m] for m in conv_mean[exp].keys()], pd.Index([m for m in conv_mean[exp].keys()], name='model'), coords='minimal')
conv_mean_ds = xr.concat([conv_dict[exp] for exp in conv_dict.keys()], pd.Index([exp for exp in conv_dict.keys()], name='experiment'), coords='minimal')


In [ ]:

def reindex_df(df, weight_col):
    """expand the dataframe to prepare for resampling
    result is 1 row per count per sample"""
    df = df.reindex(df.index.repeat(df[weight_col]))
    df.reset_index(drop=True, inplace=True)
    return(df)


data = {'mean': conv_mean_ds.sel(experiment = '1000gtc').mean(dim = ['pulse_type']).sel(s = slice(60,80)).mean(dim = ['s']).values,
        'model': conv_mean_ds.model.values,
        'count': [3, 3, 3, 3, 12, 12, 12, 12, 4, 4, 4]}
df_1000gtc_emulator = pd.DataFrame(data)
df_1000gtc_emulator = reindex_df(df_1000gtc_emulator, weight_col = 'count')

data = {'mean': conv_mean_ds.sel(experiment = '1pct').mean(dim = ['pulse_type']).sel(s = slice(60,80)).mean(dim = ['s']).values,
        'model': conv_mean_ds.model.values,
        'count': [3, 3, 3, 3, 12, 12, 12, 12, 4, 4, 4]}
df_1pct_emulator = pd.DataFrame(data)
df_1pct_emulator = reindex_df(df_1pct_emulator, weight_col = 'count')


data = {'mean': ds_dif.where(ds_dif['model'].isin(model_weights.model.values), drop = True).sel(experiment = '1pct').sel(s = slice(60,80)).weighted(A).mean(dim = ['s', 'lon','lat'])['tas'].values,
        'model': ds_dif.where(ds_dif['model'].isin(model_weights.model.values), drop = True).sel(experiment = '1pct').model.values,
        'count': [3, 3, 3, 3, 12, 12, 12, 12, 4, 4, 4]}

df_1pct_model = pd.DataFrame(data)
df_1pct_model = reindex_df(df_1pct_model, weight_col = 'count')


#######ZEC############

data = {'mean': (conv_mean_ds.sel(experiment = '1000gtc').where(conv_mean_ds['model'].isin(list(model_run_1pct1000gtc_dict.keys())), drop = True).mean(dim = 'pulse_type').sel(
                                                                s = slice(80-10, 80+10)).mean(dim = 's').weighted(A).mean(dim = ['lat','lon']) - 
                    conv_mean_ds.sel(experiment = '1000gtc').where(conv_mean_ds['model'].isin(list(model_run_1pct1000gtc_dict.keys())), drop = True).mean(dim = 'pulse_type').sel(s = 65).weighted(A).mean(dim = ['lat','lon'])).values,
        'model': conv_mean_ds.where(conv_mean_ds['model'].isin(list(model_run_1pct1000gtc_dict.keys())), drop = True).model.values,
        'count': [3, 3, 3, 3, 12, 12, 12, 4, 4, 4]}

df_zec_emulator_model = pd.DataFrame(data)
df_zec_emulator_model = reindex_df(df_zec_emulator_model, weight_col = 'count')




data = {'mean': (ds_dif.where(ds_dif['model'].isin(list(model_run_1pct1000gtc_dict.keys())), drop = True).sel(experiment = '1000gtc').sel(
                                                                s = slice(80-10, 80+10)).mean(dim = 's').weighted(A).mean(dim = ['lat','lon']) - 
                    ds_dif.where(ds_dif['model'].isin(list(model_run_1pct1000gtc_dict.keys())), drop = True).sel(experiment = '1000gtc').sel(s = 65).weighted(A).mean(dim = ['lat','lon']))['tas'].values,
        
        'model': ds_dif.where(ds_dif['model'].isin(list(model_run_1pct1000gtc_dict.keys())), drop = True).sel(experiment = '1000gtc').model.values,
        'count': [3, 3, 3, 3, 12, 12, 12, 4, 4, 4]}

df_zec_cmip_model = pd.DataFrame(data)
df_zec_cmip_model = reindex_df(df_zec_cmip_model, weight_col = 'count')

## Spatial Analysis

In [ ]:
%%time

GF = G_ds

conv = {}
for exp in ['1000gtc','1pct']:
    print(exp)
    conv[exp] = {}
    if exp == '1000gtc':
        model_list = ['UKESM1_r1', 'UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2',]
    elif exp == '1pct':
        model_list = ['UKESM1_r1', 'UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2', 'GFDL', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2',]
    for m1 in model_list:
        conv[exp][m1] = {}
        for t in ['pulse','cdr']:
            if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
                m2 = 'UKESM1_r1'
            else:
                m2 = m1
            print(m1, m2)
            conv[exp][m1][t] = signal.convolve(np.array(GF.sel(model = m2, pulse_type = t).dropna(dim = 's')), 
                                               np.array(emis_profile.sel(model = m1, experiment = exp)['emis'])[~np.isnan(np.array(emis_profile.sel(model = m1, experiment = exp)['emis']))][..., None, None],
                                               'full')
            conv[exp][m1][t] = utils.np_to_xr(conv[exp][m1][t], GF.sel(model = m2, pulse_type = t), emis_profile.sel(model = m1, experiment = exp))


In [ ]:
conv_dict[exp][m]

In [37]:
#convert to dataset

conv_dict = {}
for exp in conv.keys():
    conv_dict[exp] = {}
    for m in conv[exp].keys():
        conv_dict[exp][m] = xr.concat([conv[exp][m][t] for t in conv[exp][m].keys()], pd.Index([t for t in conv[exp][m].keys()], name='pulse_type'), coords='minimal')
for exp in conv.keys():
    conv_dict[exp] = xr.concat([conv_dict[exp][m] for m in conv[exp].keys()], pd.Index([m for m in conv[exp].keys()], name='model'), coords='minimal')
conv_ds = xr.concat([conv_dict[exp] for exp in conv_dict.keys()], pd.Index([exp for exp in conv_dict.keys()], name='experiment'), coords='minimal')


MemoryError: Unable to allocate 5.85 GiB for an array with shape (11, 2, 551, 180, 360) and data type float64

# Save out

In [ ]:
conv_mean_ds.to_netcdf('Outputs/conv_mean_ds.nc4')

conv_ds.to_netcdf('Outputs/conv_ds.nc4')

emis_profile.to_netcdf('Outputs/emis_profile.nc4')

ds_dif.to_netcdf('Outputs/ds_dif.nc4')

ds_pictrl.to_netcdf('Outputs/ds_pictrl.nc4')